# Experimental Design, Model Evaluation, and Grid Search

Following (mostly) Python Machine Learning 3rd (Raschka) Chapter 6

## Agenda

- Variability in splitting data.  
- Holdout method.  
- Cross-validation.  
- Learning curves.  
- Validation curves.  
- Grid search.  
- Class imbalance.  


## Resources
[Model Evaluation, Model Selection, and Algorithm Selection in Machine Learning, Raschka](https://arxiv.org/abs/1811.12808)
<br>[SMOTE](https://arxiv.org/pdf/1106.1813.pdf)
<br>[Evaluation: from precision, recall and F-measure to ROC, informedness, markedness and correlation](https://arxiv.org/abs/2010.16061)
<br>[scikit-learn Model Selection](https://scikit-learn.org/stable/model_selection.html)


# Training/Test Split
This is how we've set-up our examples so far. We'll use the [breast cancer dataset from scikit-learn.](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer

bc = load_breast_cancer()

X = bc['data']
y = bc['target']
X_names = bc['feature_names']

bcDf = pd.concat([pd.DataFrame(X),pd.DataFrame(y)], axis=1)
bcDf.columns = list(X_names) + ['target']

pd.set_option('display.max_columns', 50)
bcDf.head()

## Check the numerical summaries of this data:

In [ ]:
bcDf.describe().T

## See if there are any interesting distributions or data issues

In [ ]:
bcDf.isna().sum().sum()

- No missing values, so no need to impute.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

bcDf.hist(figsize=(14,14))
plt.show()

- Doesn't look like there are any obvious data quality issues.  
- All numerical, but on different scales, so we'll need to standardize the features.

In [ ]:
import seaborn as sns

sns.heatmap(bcDf.corr())
plt.show()

- There are a lot of correlated features.  [multicollinearity](https://en.wikipedia.org/wiki/Multicollinearity) 
- We are going to try using [principal components](https://en.wikipedia.org/wiki/Principal_component_analysis) to extract uncorrelated features.  
    - We'll take more about exactly what this is doing later.  
- We are going to extract 5, which will reduce the feature space from 30 dimensions to 5. (Arbitrary decision)
    - This is technically a suite of hyperparameters we are introducing, but this is a well researched dataset.  

## Pipeline Gameplan
- Split the data into training/test. 
- Create feature processing pipeline. 
    - Standardize features. 
    - Extract 5 PCA components. (arbitrary decision; could evaluate)
- Fit a Logistic Regression model.  
- Evaluate accuracy (relatively balanced dataset so accuracy is okay).  

<img src='./diagrams/06_01.png' style='width: 700px;'>

[Image source: Python Machine Learning 3rd Edition, Raschka](https://github.com/rasbt/python-machine-learning-book-3rd-edition/tree/master/ch06)

#### Benefits of pipelines
- Chains together feature processing and fitting steps.  
- No separate feature transformation fits for test data.  
- Plays nice with many of the more robust evaluation options.  

## Split the data

In [ ]:
from sklearn.model_selection import train_test_split

def create_splits(X, y):
    return train_test_split(X, y, test_size=0.20)

X_train, X_test, y_train, y_test = create_splits(X, y)

print(f'Training sample: {X_train.shape[0]:,}')
print(f'Test sample: {X_test.shape[0]:,}')


In [ ]:
sum(y)/len(y)

In [ ]:
sum(y_train)/len(y_train)

In [ ]:
sum(y_test)/len(y_test)

## Create our pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

def generate_estimates(x, y, comp=5):
    
    modeling_pipeline = Pipeline([
        ('scaling', StandardScaler()),
        ('pca', PCA(n_components=comp)),
        ('model', LogisticRegression(penalty=None))
    ])

    return modeling_pipeline.fit(x, y)

m = generate_estimates(X_train, y_train)
m

## Make our predictions on the test set and determine performance estimates

In [ ]:
from sklearn.metrics import confusion_matrix

y_test_pred = m.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))

In [ ]:
from sklearn.metrics import roc_curve

def generate_probs(X, model):
    return model.predict_proba(X)[:, 1]

def generate_roc(y, probs):
    fpr, tpr, _ = roc_curve(y, probs)
    return fpr, tpr
    
fpr_test, tpr_test = generate_roc(y_test, generate_probs(X_test, m))
fpr_train, tpr_train = generate_roc(y_train, generate_probs(X_train, m))

plt.plot(fpr_test, tpr_test,'-r')
plt.plot(fpr_train, tpr_train,'-b')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(['Test','Training'])
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score

print("training roc score", roc_auc_score(y_train, generate_probs(X_train, m)))
print("test roc score", roc_auc_score(y_test, generate_probs(X_test, m)))

# Why is our performance estimate potentially flawed?
- How representative is the test dataset?  
- Why did we choose 20% to hold for the test set?  
- Feature sets in our partitions follow the same distribution?  
- Did we try any additional settings or parameter combinations?  
- We don't really have a baseline to compare our results against.  

## What type of variation can we expect in the test data set?

In [ ]:
samples = 1000
trainingMeans = []
testMeans = []

i = 0
while i < samples:
    X_train, X_test, y_train, y_test = create_splits(X, y)
    trainingMeans.append(np.mean(y_train))
    testMeans.append(np.mean(y_test))
    i += 1

plt.hist(trainingMeans)
plt.xlim(0.4, 0.8)
plt.title('Percent of Positive Classes in Training Data')
plt.show()

plt.hist(testMeans)
plt.xlim(0.4, 0.8)
plt.title('Percent of Positive Classes in Test Data')
plt.show()

- Smaller dataset will have more vulnerability to sampling issues.  
- What if our data set was bigger?  

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# population for our illustration
population = np.random.binomial(1, 0.5, size=1000000)
population

In [ ]:
pd.Series(population).value_counts()

In [ ]:
population.mean()

#### Let's draw samples at varying sample sizes to see how representative sample means would be from a large distribution.

In [ ]:
from collections import defaultdict
reps = 100
sample_sizes = [5, 10, 50, 100, 250, 500, 1000, 2500, 5000, 10000, 100000, 200000]
means = defaultdict(list)

for rep in range(reps):
    for ss in sample_sizes:
        means[ss].append(np.random.choice(a=population, size=ss, replace=False).mean())
        
meansDf = pd.DataFrame.from_dict(means)
meansDf #.head()

In [ ]:
meansDf.describe().iloc[3:, :]

In [ ]:
meansDf.quantile(0.50)
plt.plot(meansDf.quantile(0.50))
plt.xscale('log')

In [ ]:
for i in [0.05, 0.1, 0.25, 0.50, 0.75, 0.9, 0.95, 1]:
    if i == 0.5:
        plt.plot(meansDf.quantile(i), '-b')
    else:
        plt.plot(meansDf.quantile(i), '--r')
plt.xscale('log')
plt.title('Percentiles of Sampling Distribution')
plt.xlabel(f'Sample Sizes {sample_sizes}')
plt.show()

- Small samples are going to be much more at risk for sampling variations.  
- It is not advisable to use a simple training/test split for small datasets.  
    - It's very likely your test set won't mirror your training.  
    - Neither may mirror the "population".  
- For larger datasets, choosing a smaller percentage for test can be okay.  
    - The number of samples held will be hopefully very large, and have less risk to sampling variation.  

### Rascha on Choosing an appropriate ratio for partitioning a dataset into training and test dates
From page 124 of *Python Machine Learning 3rd Edition, Raschka*:
> If we are dividing a dataset into training and test datasets, we have to keep in mind that we are withholding valuable information that the learning algorithm could benefit from.<br><br>Thus, we don't want to allocate too much information to the test set.<br><br>However, the smaller the test set, the more inaccurate the estimation of the generalization error.<br><br>Dividing a dataset into training and test datasets is all about balancing this tradeoff.<br><br>...<br><br>Moreover, instead of discarding the allocated test data after model training and evaluation, it is a common practice to retrain a classifier on the entire dataset, as it can improve the predictive performance of our model.

## Strengths
- Quick guage of performance.  
- Okay if looking for quick interpretability.


## Weaknesses
- Prone to overfitting and not ideal for tuning hyperparameters. 
- Many don't partition the training into training and validation, which isn't a good practice.  
- There could be other issues with the splits that affect the comparability and would lead to poor results in the real world. The performance estimate is going to be sensitive to how the partition was done. 

## Other issues to be aware
- Is the historical data representative of the current data generating process?  
- Have there been structure shifts in the data over time? (leakage)  
- Rare event issues and sampling

# Better: Holdout Method
This is for larger datasets, where we don't need to worry about sampling variation risk. We are going to pretend the breast cancer data is "big" enough for this to be a valid approach.

- Split the data into training and test.  
> **Develop and train our models using the training partition and estimate general performance on the test set.**  
**PRETEND THE TEST SET DOESN'T EXIST WHILE MODELING OR PERFORMING PREPROCESSING. IDEAS WHY?**
- Split the training into training and validation.  
> **If we train our models on the entire training dataset, we are going to be at risk for overfitting? Why?** 
- Training data is where we'll be creating models and those will be evaluated on the validation data. The test set is for final validation and check that we didn't overfit on the validation set.

<img src='./diagrams/training-validation-test-data-set.png' style='width: 600px;'>

[Image source: Raschka, Chapter 6, Page 196](https://github.com/rasbt/python-machine-learning-book-3rd-edition/tree/master/ch06)

## Chaining train_test_split

In [ ]:
from sklearn.model_selection import train_test_split

def create_holdout_splits(X, y):
    x_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    X_train, X_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)
    return X_train, X_val, X_test, y_train, y_val, y_test

X_train, X_val, X_test, y_train, y_val, y_test = create_holdout_splits(X, y)

print(f'Training sample: {X_train.shape[0]:,}')
print(f'Validation sample: {X_val.shape[0]:,}')
print(f'Test sample: {X_test.shape[0]:,}')

## Compare and evaluate models
Let's see how the PCA model to a model that uses the full feature set.

### Create pipeline we can use for feature transforms and prediction.
- Might as well check our assumption on using 5 components.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

def generate_estimates(use_pca = False, comp=None):
    if use_pca:
        modeling_pipeline = Pipeline([
            ('scaling', StandardScaler()),
             ('pca', PCA(n_components=comp)),
             ('model', LogisticRegression(penalty=None))
            ]
        )
    else:
        modeling_pipeline = Pipeline([
            ('scaling', StandardScaler()),
             ('model', LogisticRegression(penalty=None))
            ]
        )

    return modeling_pipeline

m_rawFeatures = generate_estimates(use_pca=False).fit(X_train, y_train)
print('raw features:', m_rawFeatures)

m_pca_models = {}
for i in range(1,6):
    m_pca_models[i] = generate_estimates(use_pca=True, comp=i).fit(X_train, y_train)
    print(f'pca{i}:', m_pca_models[i])
    
print('Models fitted')

### Compare confusion matrices

In [ ]:
from sklearn.metrics import confusion_matrix

y_val_rawFeatures = m_rawFeatures.predict(X_val)

print('Using raw features:')
print(confusion_matrix(y_val, y_val_rawFeatures))
print('----------------------')
y_val_pca = {}
for i in range(1,6):
    y_val_pca[i] = m_pca_models[i].predict(X_val)
    print(f'Using PCA{i}:')
    print(confusion_matrix(y_val, y_val_pca[i]))

- Looks like the PCA model performed better and maybe we didn't need all 5 of those components.

### Check the ROC curves

In [ ]:
from sklearn.metrics import roc_curve

def generate_probs(X, model):
    return model.predict_proba(X)[:, 1]

def generate_roc(y, probs):
    fpr, tpr, _ = roc_curve(y, probs)
    return fpr, tpr
    
fpr_val_rawFeatures, tpr_val_rawFeatures = generate_roc(y_val,
                                                        generate_probs(X_val, model=m_rawFeatures))


fpr_val_pca = {}
tpr_val_pca = {}
for i in range(1,6):
    fpr_val_pca[i], tpr_val_pca[i] = generate_roc(y_val,
                                          generate_probs(X_val, model=m_pca_models[i]))


plt.plot(fpr_val_rawFeatures, tpr_val_rawFeatures,'-r')
for i in range(1,6):
    plt.plot(fpr_val_pca[i], tpr_val_pca[i],'-b')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(['Raw Features (No Regularization)','PCA'])
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.show()

- Looks like the ROC curves overlap themselves for each component.  
- Using PCA performed significantly better in the validation data.  
- Now check to see if it continues in the test data.  


In [ ]:
fpr_test_rawFeatures, tpr_test_rawFeatures = generate_roc(y_test,
                                                        generate_probs(X_test, model=m_rawFeatures))
print(f'Using raw features:')
print(confusion_matrix(y_test, m_rawFeatures.predict(X_test)))
print(roc_auc_score(y_test, generate_probs(X_test, m_rawFeatures)))
print('\n')

# create false/true positive rate curves
fpr_test_pca = {}
tpr_test_pca = {}
for i in range(1,6):
    conf_matrix = m_pca_models[i].predict(X_test)
    print(f'Using PCA{i}:')
    print(confusion_matrix(y_test, conf_matrix))
    
    probs = generate_probs(X_test, model=m_pca_models[i])
    fpr_test_pca[i], tpr_test_pca[i] = generate_roc(y_test, probs)
    print(roc_auc_score(y_test, probs))
    
    print('\n')

# plot the ROC curve
plt.plot(fpr_test_rawFeatures, tpr_test_rawFeatures,'-r')

for i in range(1,6):
    plt.plot(fpr_test_pca[i], tpr_test_pca[i],'-b')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(['Raw Features (No Regularization)','PCA'])
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.show()

- Looks like the performance was consistent with what we observed in the validation dataset.
- Which model should be choose in this case?

# Cross-validation - Best option for small datasets

<img src='./diagrams/06_03.png' style='width: 600px;'>

[Image source: Python Machine Learning 3rd Edition, Figure 6.3](https://github.com/rasbt/python-machine-learning-book-3rd-edition/tree/master/ch06)

#### Basic mechanics:  
- Split the data into training and test. 
- The training data will be divided into *k* folds.  
    - Each folder will use a different partition for the validation data.  
- The models will be run on each fold.  
- Performance estimate will be taken with the median or mean score.  
    - Need to define what metric you are optimizing toward.  

You could do this with a loop, but scikit-learn has this built-in.

Let's get our data again:

In [ ]:
from sklearn.model_selection import train_test_split

def create_splits(X, y):
    return train_test_split(X, y, test_size=0.20)

X_train, X_test, y_train, y_test = create_splits(X, y)

print(f'Training sample: {X_train.shape[0]:,}')
print(f'Test sample: {X_test.shape[0]:,}')

#### Set-up the same pipeline as before and run it through the function
Things to consider:  
- Need an estimator (classifer/regression), which a pipeline satisfies if the last step is a model.  
- This accepts multiple metrics, so you'll need to determine which one is most appropriate.

[List of metrics accepted](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

def generate_estimates(comp=5):
    
    modeling_pipeline = Pipeline([
        ('scaling', StandardScaler()),
        ('pca', PCA(n_components=5)),
        ('model', LogisticRegression(penalty=None))
    ])

    return modeling_pipeline

from sklearn.model_selection import cross_validate

clf = generate_estimates()
cv_results = cross_validate(clf, X_train, y_train, 
                            scoring=['accuracy', 'recall', 'precision', 'f1_macro', 'roc_auc'], cv=5)
cv_results

#### What does this return?
- How long it took the model to fit. This will be more important with larger datasets and more complex models.  
- How long it took the model to score the validation set in the fold.  
- The metrics.  

It returns an array so you can look at the distribution and/or central tendency of the model.

In [ ]:
for k in cv_results.keys():
    print(f'{k}: {cv_results[k].mean():.4f} (+/- {cv_results[k].std():.4f})')

## Solved the sampling variation issue, now can compare between models with better certainty


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

def generate_estimates(use_pca = False, comp=None):
    if use_pca:
        modeling_pipeline = Pipeline([
            ('scaling', StandardScaler()),
            ('pca', PCA(n_components=5)),
            ('model', LogisticRegression(penalty=None))
        ])
    else:
        modeling_pipeline = Pipeline([
            ('scaling', StandardScaler()),
            ('model', LogisticRegression())
        ])

    return modeling_pipeline

from sklearn.model_selection import cross_validate

pca_clf = generate_estimates(use_pca=True, comp=5)
nopca_clf = generate_estimates()

pca_cv_results = cross_validate(pca_clf, X_train, y_train, scoring=['accuracy', 'recall', 'precision', 'f1_macro', 'roc_auc'], cv=5)
pca_cv_results

In [ ]:
nopca_cv_results = cross_validate(nopca_clf, X_train, y_train, scoring=['accuracy', 'recall', 'precision', 'f1_macro', 'roc_auc'], cv=5)
nopca_cv_results

In [ ]:
a1 = pca_cv_results['test_accuracy'].mean()
a2 = nopca_cv_results['test_accuracy'].mean()

plt.plot(pca_cv_results['test_accuracy'])
plt.plot(nopca_cv_results['test_accuracy'])
plt.legend([f'PCA (mean={a1:.2f})',f'No PCA (mean={a2:.2f})'])
plt.title('Accuracy on Validation')
plt.show()

In [ ]:
a1 = pca_cv_results['test_f1_macro'].mean()
a2 = nopca_cv_results['test_f1_macro'].mean()

plt.plot(pca_cv_results['test_f1_macro'])
plt.plot(nopca_cv_results['test_f1_macro'])
plt.legend([f'PCA (mean={a1:.2f})',f'No PCA (mean={a2:.2f})'])
plt.title('F1-Score on Validation')
plt.show()

> What looks like the better option?

## Cross-validation not just for classification tasks
- Same concept for regression, the scoring metrics will be different.  
- Recall our California housing dataset:

In [ ]:
from sklearn.datasets import fetch_california_housing

ch = fetch_california_housing()
ch.keys()

In [ ]:
X_housing = pd.DataFrame(ch['data'], columns=list(ch['feature_names']))
X_housing.head(2)

In [ ]:
y_housing = ch['target']

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

def generate_estimates():
    
    modeling_pipeline = Pipeline([
        ('scaling', StandardScaler()),
        ('model', LinearRegression())
    ])

    return modeling_pipeline

from sklearn.model_selection import cross_validate

reg = generate_estimates()
cv_results = cross_validate(reg, X_housing, y_housing, scoring=['r2'], cv=10)
cv_results

In [ ]:
plt.hist(cv_results['test_r2'])
plt.title('$R^2$ Distribution from Cross-validation')
plt.show()

> Looks like the parameters being learned are highly influenced by the training data for California housing. (overfit)

## But what about setting hyperparameters?
How to determine how many components we may want to use?

### General comments on hyperparameters:
- These aren't learned by the model, they are selected by the analyst.  
- You need to learn the best combination by experimenting across the search space.  
- If you have a large dataset you can evaluate these in the validation dataset.  
- If you have a smaller dataset you can evaluate these using cross validation.  

Consider how large your search space can be:
- For regularization, the $C$ parameter in Logistic Regression could span from near zero to approaching infinity.  
- For hyperparameters with that type of bounds, you'll likely want to start with a handful of values that are spacing on a logarithmic scale, e.g. $C \in (0.001, 0.01, 0.1, 1, 10, 100, 1000)$.  
- If you have other hyperparameters, e.g., the specific solver for Logistic Regression, you'll need to evaulate each solver at each regularization level:  

$$\begin{equation*}
PE_{solver,C} = 
\begin{pmatrix}
PE_{newton-cg,0.001} & PE_{newton-cg,0.01} & \cdots & PE_{newton-cg,1000} \\
PE_{lbfgs,0.001} & PE_{lbfgs,0.01} & \cdots & PE_{lbfgs,1000} \\
\vdots  & \vdots  & \ddots & \vdots  \\
PE_{saga,0.001} & PE_{saga,0.01} & \cdots & PE_{saga,1000} 
\end{pmatrix}
\end{equation*}
$$

- So you if you have 7 different regularization strengths and 5 different solvers, you'll be running $7*5=35$ models. If you use 10 cross-validation folds, that will be 350 models (hence smaller datasets for CV).  
- And making it more complicated, you may have multiple metrics (e.g., precision vs. recall).  

> When you find the "best" set of hyperparameters, you'll want to explore the nearby space in more detail.

- If you find $C=10$ as the "best", you might want to try another round with $C \in (7,8,9,10,11,12,13)$.  

> Finding the "perfect" set of hyperparameters is likely impossible. The search space is generally going to be too large.

- Random search of Bayesian hyperparameter optimization can help in those situations.
- In random search you'll provide a distribution of values instead of discrete values.  
- Optimization-based searches will try to make intelligent choices based on past explorations.  

# Validation Curves
- You can use these to visualize seeing the differences in different parameters.  
- scikit-learn uses a varient on k-fold cross-validation to plot the distribution of metrics for different parameter values.  
- We can plot the range of accuracies we observe for many folds across different settings of the parameter.

This is using the breast cancer dataset. We are going to vary $C$ to see its effect. Some of this code is borrowed from page 205-206 of Python Machine Learning 3rd Edition.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

lg_pipe = modeling_pipeline = Pipeline([
         ('scaler', StandardScaler()),
         ('pca', PCA(n_components=5)),
         ('logreg', LogisticRegression(penalty='l2'))
        ]
    )

from sklearn.model_selection import validation_curve

param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]

training_scores, test_scores = validation_curve(estimator=lg_pipe, X=X_train, y=y_train,
                                               param_name='logreg__C', 
                                               param_range=param_range, 
                                               cv=10)

train_mean = np.mean(training_scores, axis=1)
train_std = np.std(training_scores, axis=1)

test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

plt.plot(param_range, train_mean, color='blue', marker='o', label='Training Accuracy')
plt.fill_between(param_range, train_mean + train_std, train_mean - train_std, alpha=0.2, color='blue')

plt.plot(param_range, test_mean, color='orange', marker='o', label='Test Accuracy')
plt.fill_between(param_range, test_mean + test_std, test_mean - test_std, alpha=0.2, color='orange')

plt.xscale('log')
plt.xlabel('$C$')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

> What does this suggest as a parameter for $C$?

# Grid Search

<img src='./diagrams/grid_search_workflow.png' style='width: 500px;'>

[Image source](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation)

Examples:
- [Digits dataset](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html#sphx-glr-auto-examples-model-selection-plot-grid-search-digits-py)  
- [Text extraction](https://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html#sphx-glr-auto-examples-model-selection-grid-search-text-feature-extraction-py)  
- [Multiple metric example](https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html#sphx-glr-auto-examples-model-selection-plot-multi-metric-evaluation-py)

On the breast cancer dataset:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

lg_pipe = modeling_pipeline = Pipeline([
        ('scaler', StandardScaler()),
         ('pca', PCA(n_components=5)),
         ('logreg', LogisticRegression(penalty='l2', solver='liblinear'))
        ]
    )

from sklearn.model_selection import GridSearchCV

param_grid = [
  {
    'logreg__penalty': ['l1', 'l2'],
    'logreg__C': [1, 10, 100, 1000], 
    'pca__n_components': [1,2,3,4,5,10,15]
  }
 ]

gcv_results = GridSearchCV(estimator=lg_pipe, param_grid=param_grid, scoring='accuracy')
gcv_results = gcv_results.fit(X_train, y_train)

> This outputs a dictionary with the results:

In [ ]:
gcv_results.cv_results_.keys()

> And a series of summarys for the best fit

In [ ]:
gcv_results.best_score_

In [ ]:
gcv_results.best_params_

In [ ]:
gcv_results.best_estimator_

> After you find the "best" hyperparameters, you'll retrain your training data using those and then evaluate the test data using that model. There's an option in GridSearchCV to do this automatically.

You could use multiple estimators, but it'll get a little complicated, see below for example:

```python
   from sklearn.base import BaseEstimator
    from sklearn.model_selection import GridSearchCV
    
    class DummyEstimator(BaseEstimator):
        def fit(self): pass
        def score(self): pass
        
    # Create a pipeline
    pipe = Pipeline([('clf', DummyEstimator())]) # Placeholder Estimator
    
    # Candidate learning algorithms and their hyperparameters
    search_space = [{'clf': [LogisticRegression()], # Actual Estimator
                     'clf__penalty': ['l1', 'l2'],
                     'clf__C': np.logspace(0, 4, 10)},
                    
                    {'clf': [DecisionTreeClassifier()],  # Actual Estimator
                     'clf__criterion': ['gini', 'entropy']}]
    
    
    # Create grid search 
    gs = GridSearchCV(pipe, search_space)
```

# Evaluation Wrap-up

<img src='./diagrams/model-eval-conclusions.jpg' style='width: 600px;'>

[Image source: *Model Evaluation, Model Selection, and Algorithm Selection in Machine Learning, Raschka*](https://sebastianraschka.com/blog/2018/model-evaluation-selection-part4.html)

# Unbalanced Classes

Consider the credit from last week:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

credit = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/log_reg/Default.csv', index_col=0)
credit.head()

> Remember we had interesting distributions with our numerical variables.

In [ ]:
credit.hist(bins=50)
plt.show()

In [ ]:
credit['balance2income'] = credit['balance']/credit['income']

credit['balance2income'].hist(bins=100)
plt.title('Balance-Income Ratio')
plt.show()

In [ ]:
credit['default'].value_counts().plot.barh()
plt.title('Default are relatively rare...', loc='left')
plt.show()

### Recall we didn't have the best results with a straightforward logistic regression

> Will try to add some additional features to capture some of the mixture distributions and the truncation of the credit balance.

We can add these prior to processing since they are only considering the example row.

In [ ]:
credit['balance_student_int'] = np.where(credit['student']=='Yes', credit['balance'], 0)
credit['income_student_int'] = np.where(credit['student']=='Yes', credit['income'], 0)
credit['zero_balance'] = np.where(credit['balance'] == 0, 'Yes', 'No')

credit.head()

### We'll do Cross Validation, but still need to split the data into training and test

In [ ]:
from sklearn.model_selection import train_test_split

creditFeatures = [x for x in credit.columns if x != 'default']
y = np.where(credit['default'] == 'Yes', 1, 0)

X_train, X_test, y_train, y_test = train_test_split(credit[creditFeatures], y, test_size=0.20)

print(f'Training examples: {X_train.shape[0]:,}')
print(f'Test examples: {X_test.shape[0]:,}')

In [ ]:
y_train.mean(), y_test.mean()

#### Create a modeling pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

nums = ['balance2income', 'balance', 'income', 'balance_student_int', 'income_student_int']
ohes = ['student', 'zero_balance']

processing_pipeline = ColumnTransformer(transformers=[
    ('numscaling', StandardScaler(), nums),
    ('dummys', OneHotEncoder(drop='first'), ohes)]
)

modeling_pipeline = Pipeline([
    ('data_processing', processing_pipeline),
    ('logreg', LogisticRegression())]
)

### Run a baseline model

In [ ]:
creditbase = modeling_pipeline.fit(X_train, y_train)
base_p = creditbase.predict(X_test)
base_pr = creditbase.predict_proba(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, base_p))

### Define search space
> We'll try using weights to try to account for the unbalanced distribution of defaults.

In [ ]:
param_grid = [
  {
    'logreg__class_weight': [None, 'balanced'], 
   'logreg__C':[0.01, 0.1, 1, 10, 100]}
 ]

### Run the experiment
> Let's look for high-recall, since default could be very expensive.

In [ ]:
gcv_results = GridSearchCV(estimator=modeling_pipeline, param_grid=param_grid, scoring='recall', refit=True)
gcv_results = gcv_results.fit(X_train, y_train)

In [ ]:
gcv_results.best_estimator_

In [ ]:
gcv_results.best_params_

### Determine how this performs on the test data

In [ ]:
y_testp = gcv_results.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_testp))

### Did weighting help?
- Recall went from 0.36 to 0.92 for detecting the default.  
- Precision went from 0.69 to 0.17 for detecting the default.  
> We detected more of the defaults, with the trade-off of more false-positives. Depending what the cost of those is - which you'd need to talk with a SME about - this might be a preferred model.

In [ ]:
from sklearn.metrics import roc_curve

y_testpr = gcv_results.predict_proba(X_test)

def generate_roc(y, probs):
    fpr, tpr, _ = roc_curve(y, probs)
    return fpr, tpr
    
fpr_wgt, tpr_wgt = generate_roc(y_test, y_testpr[:,1])
fpr_base, tpr_base = generate_roc(y_test, base_pr[:,1])

plt.plot(fpr_wgt, tpr_wgt,'-r')
plt.plot(fpr_base, tpr_base,'--b')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(['Weighted Model','Unweighted Model'])
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.show()

> Weighted model has essentially the same ROC curve as the unweighted model in this case.

In [ ]:
plt.barh(creditbase['data_processing'].get_feature_names_out(),creditbase['logreg'].coef_[0])
plt.barh(creditbase['data_processing'].get_feature_names_out(), gcv_results.best_estimator_['logreg'].coef_[0], alpha=0.5)
plt.legend(['Unweighted Coefficients', 'Weighted Coefficients'])
plt.show()

## Resampling and Generating new data

### Resampling. 
If weighting isn't supporting, you can resample the training data.  
- This gets complicated since you want to evaluate based on the original distribution.  
- GridSearchCV won't support this well since the validation dataset is split from the training data.  
- You could use loops for this, but you need to be careful to make sure the performance estimate is based on the `actual` distribution.

### Generating new data.  
- The more complicated, probably less ROI option, is generating new data.  
-  Synthetic Minority Over-sampling Technique(SMOTE) is a popular technique, if needed. [See this paper for a description](https://arxiv.org/pdf/1106.1813.pdf)

> There is another library called `imbalanced-learn` that has methods specifically designed for these types of problems as well.

# Using classifiers to determine dataset bias 
- We shouldn't be able to predict whether an example in the training or test set.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()

iX_train, iX_test, iy_train, iy_test = train_test_split(iris['data'], 
                                                        iris['target'], 
                                                        shuffle=True)

Creating a new label, whether included in the training or test set and that will be our new target.

In [ ]:
inTraining = np.ones((iX_train.shape[0], 1))
inTest = np.zeros((iX_test.shape[0], 1))

irisTarget = np.append(inTraining, inTest, axis=0).reshape(-1)
irisTraining = np.append(iX_train, iX_test, axis=0)

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(class_weight='balanced')
clf = clf.fit(irisTraining, irisTarget)
preds = clf.predict(irisTraining)

from sklearn.metrics import confusion_matrix
confusion_matrix(irisTarget, preds)

In [ ]:
from sklearn.dummy import DummyClassifier

dumdum = DummyClassifier(strategy='uniform')
dumdum = dumdum.fit(irisTraining, irisTarget)
dumPreds = dumdum.predict(irisTraining)

confusion_matrix(irisTarget, dumPreds)

Those are pretty close, so we would have confidence the test and training data is nearly identical if we were making a classifier.

# Readings
[Raschkas's Lecture](https://github.com/rasbt/stat479-machine-learning-fs19/blob/master/11_eval4-algo/11-eval4-algo__slides.pdf)
<br>[Full paper: Model Evaluation, Model Selection, and Algorithm Selection in Machine Learning, Raschka](https://arxiv.org/abs/1811.12808)
<br>[Evaluation: From Precision, Recall and F-Factor to ROC, Informedness, Markedness & Correlation](https://arxiv.org/abs/2010.16061)